#  1. 二分查找基本思想

二分查找的基本思想是将含有 n 个元素的数组（升序排列）分成大致相等的两部分，取 $a[n/2]$与 x 做比较：
1. 如果 $x == a[n/2]$ , 则找到 x , 算法中止
2. 如果 $x < a[n/2]$ , 则只需要在数组 a 的左半部分继续搜索 x
3. 如果 $a[n/2] < x$ , 则只需要在数组 a 的右半部分继续搜索 x

In [2]:
from typing import List
def binary_search(nums: List[int], x: int) -> int:
    
    left = 0 
    right = len(nums)
    loop_cnt = 0
    while left < right:
        print(f"{loop_cnt} loop size: {right - left + 1}")
        
        mid = left + ((right - left) >> 1)
        if nums[mid] == x:
            return mid
        elif nums[mid] < x:
            left = mid + 1
        else:
            right = mid
        loop_cnt += 1
        
    return left if nums[left] == x else -1        

In [3]:
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9]
index = binary_search(arr, 9)
print(index)

0 loop size: 10
1 loop size: 5
2 loop size: 2
8


# 2. 二分查找时间复杂度

时间复杂度即是 while 循环的次数。
总共有 n 个元素，渐渐跟下去就是 $n, \frac{n}{2} , \frac{n}{2}, \ldots, \frac{n}{2^k}$（接下来操作元素的剩余个数），其中 k 就是循环的次数. 由于 $\frac{n}{2^k}$取整后 >= 1, 
\begin{equation}
\frac{n}{2^k} = 1 \rightarrow k = \log_{2} n
\end{equation}
所以时间复杂度可以表示 $O(n)=O(\log_2 n)$ 或者 $O(n)=O(\log n)$

# 3. 二分查找模板

一般而言，当一个题目出现以下特性时，我们就应该立即联想到它可能需要使用二分查找：

* **待查找的数组有序或者部分有序**
* 要求时间复杂度低于 O(n)，或者直接要求时间复杂度为 O(log n)

二分查找有很多种变体，使用时需要注意查找条件，判断条件和左右边界的更新方式，三者配合不好就很容易出现死循环或者遗漏区域，本篇中我们将介绍常见的几种查找方式的模板代码，包括：

* 标准的二分查找
* 二分查找左边界
* 二分查找右边界
* 二分查找左右边界
* 二分查找极值点

## 3.1 模板一：标准的二分查找

首先，标准二分查找的适用场景是：**数组元素有序且不重复**
我们以 “搜索一个数，如果存在，返回其索引，否则返回 -1” 为例，给出标准二分查找的模板：

In [23]:
def binarySearch(nums: List[int], target: int) -> int:
    left = 0
    right = len(nums) - 1
    
    while left <= right:
        mid = left + ((right - left) >> 1)
        if nums[mid] == target:
            return mid
        elif nums[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1

arr = [1, 2, 3, 4, 5, 6, 7, 8, 9]
index = binarySearch(arr, 3)
print(index)      

2


1. **为什么 while 循环的条件是 <=，而不是 < ？**

因为初始化 right 的赋值是 nums.length - 1，即最后一个元素的索引，而不是 nums.length. 

在 right 是这样赋值的前提下，<= 相当于两端都是闭区间 [left, right]，< 相当于左闭右开区间 [left, right)，因为索引大小为 nums.length 是越界的。

我们这个算法中使用的是前者 [left, right] 两端都闭的区间。这个区间其实就是每次进行搜索的区间。

什么时候应该停止搜索呢？当然，找到了目标值的时候可以终止：

```
if nums[mid] == target:
    return mid
```

但如果没找到，就需要 while 循环终止，然后返回 -1。那 while 循环什么时候应该终止？**搜索区间为空的时候应该终止**，意味着你没得找了，就等于没找到嘛: 

* while(left <= right) 的终止条件是 left == right + 1，写成区间的形式就是 [right + 1, right]，或者带个具体的数字进去 [3, 2]，可见这时候区间为空，因为没有数字既大于等于 3 又小于等于 2 。所以这时候 while 循环终止是正确的，直接返回 -1 即可。

* while(left < right) 的终止条件是 left == right，写成区间的形式就是 [left, right]，或者带个具体的数字进去 [2, 2]，这时候区间非空，还有一个数 2，但此时 while 循环终止了。也就是说这区间 [2, 2] 被漏掉了，索引 2 没有被搜索，如果这时候直接返回 -1 就是错误的。当然，如果你非要用 while(left < right) 也可以，我们已经知道了出错的原因，打个补丁好了
```
return left if nums[left] == target else -1
```

2. **计算 mid 时需要防止溢出**

left + ((right -left) >> 1) 其实和 (left + right) / 2 是等价的, **这样写的目的一个是为了防止 (left + right) 出现溢出，另外用位运算替代除法提升性能.**

keft + ((right -left) >> 1) 对于目标区域长度为奇数而言，是处于正中间的，对于长度为偶数而言，是中间偏左的。因此左右边界相遇时，只会是以下两种情况：

* left/mid , right ( left, mid 指向同一个数，right 指向它的下一个数)
* left/mid/right （ left, mid, right 指向同一个数）

因为 mid 对于长度为偶数的区间总是偏左的，所以当区间长度小于等于 2 时，mid 总是和 left 在同一侧

# 3.2 模板二：二分查找边界

如果给一个数组是有序且有重复元素的数组， 比如说给你有序数组 nums = [1,2,2,2,3]， target 为 2，此算法返回的索引是 2. 但是如果我<u>想得到 target 的左侧边界，即索引 1, 或者我想得到 target 的右侧边界，即索引 3</u>，这样的话上述标准形式的二分查找算法是无法处理的。

In [24]:
nums = [1,2,2,2,3]
index = binarySearch(nums, 2)
print(index)

2


利用二分法寻找左（右）边界是二分查找的一个变体，一般适用场景可以分为两种：

1）第一种情况:

* 数组部分有序，且不包含重复元素
* 数组有序，但包含重复元素
![](image/search/bs_1.png)

2) 第二种情况:
* 数组部分有序，且包含重复元素
![](image/search/bs_2.png)

### 二分查找左边界

1) 先来看第一种情况：

* 数组有序，但包含重复元素

**既然要寻找左边界，搜索范围就需要从右边开始，不断往左边收缩**，也就是说即使我们找到了nums[mid] == target, 这个 mid 的位置也不一定就是最左侧的那个边界，我们还是要向左侧查找，也就是说，除了在 nums[mid] 偏大的时候需要收锁右边界，在 nums[mid] 等于目标值的时候，不同于标准的二分查找直接返回，我们此时仍然需要继续收缩右边界

In [8]:
def binarySearch_left_case1(nums: List[int], target: int) -> int:
    left = 0
    right = len(nums) - 1
    
    while left < right:
        mid = left + ((right - left) >> 1)
        if nums[mid] < target:
            left = mid + 1
        else: 
            # target <= nums[mid]
            right = mid
    
    return left if nums[left] == target else -1
nums = [1,2,2,2,3]
index = binarySearch_left_case1(nums, 2)
print(index)

1


与标准的二分查找不同：

1. 这里的右边界的更新是 nums[mid] >= targets，因为我们需要在找到目标值后，继续向左寻找左边界。
2. 这里的循环条件是 left < right 而不是 <=：

我们把处理到 left + 1 = right 位置上的 while 循环称为倒数第二轮循环，把处理到 left = right 也即最后一个数的 while 循环称为最后一轮循环。

因为在最后 left 与 right 相邻的时候也即 left + 1 = right 倒数第二轮循环的时候，mid 和 left 处于相同的位置(前面说过，mid 偏左)，那么，此时无论是走到 left = mid + 1 也好，还是 right = mid 也好，如果此时循环的条件是 left <= right，就会进入最后一轮循环（left = right，只剩最后一个数了），left 和 right 必定是处于同一个位置，也就是说 left, mid, right 都将指向同一个位置，对吧，其实这逻辑没有问题，标准的二分查找中也确实需要走到最后一步循环。

但是，我们这的代码和标准二分查找的代码不同：

* 如果 nums[mid] < target，left 会变成 mid + 1 从而破坏 while 条件从而循环正常终止；
* 但是，如果 nums[mid] >= target ，我们会令 right = mid，这个时候总共只有一个数了，mid 赋值给 right 并没有改变 left,mid,right 的位置，这将导致彻底进入死循环！

那既然如果走到最后一轮循环会可能导致死循环的发生，那我们只走到倒数第二轮循环即 left 和 right 相邻就好了，把最后一个数单独拉出来处理（或者说打个补丁）：

```
return left if nums[left] == target else -1
```

2）再来看第二种情况：
* 数组部分有序，且包含重复元素

同样地，我们先以查找左边界为例：

这种条件下，当我们找到目标元素的时候（nums[mid] = target），向左收缩右边界不能简单的令 right = mid，因为有重复元素的存在，这会导致我们有可能遗漏掉一部分区域。此时需要采用比较保守的方式即 right --

In [11]:
def binarySearch_left_case2(nums: List[int], target: int) -> int:
    left = 0
    right = len(nums) - 1
    
    while left < right:
        mid = left + ((right - left) >> 1)
        if nums[mid] < target:
            left = mid + 1
        elif target < nums[mid]:
            right = mid
        else: 
            # target == nums[mid]
            right -= 1
    
    return left if nums[left] == target else -1
nums = [1,2,2,2,4, 5, 8, 9, 3, 1]
index = binarySearch_left_case2(nums, 2)
print(index)

1


### 二分查找右边界

有了寻找左边界的分析之后，再来看寻找右边界就容易很多了，毕竟左右两种情况是对称的，适用场景也都是一样的，大家对称着理解就好，这里我们给出二分查找右边界的第一种情况的模板（即数组有序，但包含重复元素；数组部分有序，且不包含重复元素）：

查找左边界的代码中我们是先收缩左边界然后收缩右边界，那么相反的，查找右边界的时候我们就需要先收缩右边界然后收缩左边界。

这里大部分和寻找左边界是对称着来写的，唯独有一点需要尤其注意 —— <u>中间位置的计算变了，我们在末尾多加了 1。这样，无论对于奇数还是偶数，这个中间的位置都是偏右的。</u>


对于这个操作的理解，从对称的角度看，寻找左边界的时候，中间位置是偏左的，那寻找右边界的时候，中间位置就应该偏右，但是这显然不是根本原因。

根本原因是，在最后left和right相邻时，如果mid偏左，则left, mid指向同一个位置，right指向它们的下一个位置，在nums[left]已经等于目标值的情况下，这三个位置的值都不会更新，从而进入了死循环。所以我们应该让mid偏右，这样left就能向右移动。这也就是为什么我们之前一直强调查找条件，判断条件和左右边界的更新方式三者之间需要配合使用。

右边界的查找一般来说不会单独使用，如有需要，一般是需要同时查找左右边界。

In [9]:
def binarySearch_right_case1(nums: List[int], target: int) -> int:
    left = 0
    right = len(nums) - 1
    while left < right:
        mid = left + ((right - left) >> 1) + 1
        if target < nums[mid]:
            right = mid - 1
        else: 
            # nums[mid] <= target
            left = mid

    return right if nums[right] == target else -1
nums = [1,2,2,2,3]
index = binarySearch_right_case1(nums, 2)
print(index)

3


### 二分查找左右边界

前面我们介绍了左边界和右边界的查找，那么查找左右边界就容易很多了——只要分别查找左边界和右边界就行了。

这里给出符合第一种情况的模板，（即数组有序，但包含重复元素；数组部分有序，且不包含重复元素）


In [10]:
def binarySearch_both_case1(nums: List[int], target: int) -> int:
    left = 0
    right = len(nums) - 1
    
    while left < right:
        mid = left + ((right - left) >> 1)
        if nums[mid] < target:
            left = mid + 1
        elif target < nums[mid]:
            right = mid
        else: 
            # target == nums[mid]
            right -= 1
    
    left_idx = left if nums[left] == target else -1
        
    if left_idx != -1:
        # 如果左边界是最后一个数的下标或者该数没有重复只存在一个，
        # 那么可以直接令右边界 = 左边界
        
        if left == len(nums) - 1 or nums[left + 1] != target:
            right_idx = left
        else:
            right = len(nums) - 1
            while left < right:
                mid = left + ((right - left) >> 1) + 1
                if target < nums[mid]:
                    right = mid - 1
                else: 
                    # nums[mid] <= target
                    left = mid

            right_idx = right if nums[right] == target else -1
    return left_idx, right_idx
nums = [1,2,2,2,3]
index = binarySearch_both_case1(nums, 2)
print(index)

(1, 3)


## 3.3 模板三：二分查找极值点

标准的二分查找还有一种变体是二分查找极值点，之前我们使用 nums[mid] 去比较的时候，常常是和给定的目标值 target 比，或者和左右边界比较，**在二分查找极值点的应用中，我们是和相邻元素去比，以完成某种单调性的检测**

In [48]:
# find min value in the array
def binarySearch_min(nums: List[int]) -> int:
    left = 0
    right = len(nums) - 1
    
    while left <= right:
        mid = left + ((right - left) >> 1)
        print(f"Before: left: {left}, mid: {mid}, right: {right}")
        if nums[mid - 1] > nums[mid] and nums[mid] < nums[mid + 1]:
            return mid
        elif nums[mid] > nums[mid + 1]:
            left = mid + 1
        else:
            right = mid - 1
        print(f"After: left: {left}, mid: {mid}, right: {right}")
    return -1
nums = [9, 8, 4, 5, 7, 9, 10, 11]
index = binarySearch_min(nums)
print(index)

Before: left: 0, mid: 3, right: 7
After: left: 0, mid: 3, right: 2
Before: left: 0, mid: 1, right: 2
After: left: 2, mid: 1, right: 2
Before: left: 2, mid: 2, right: 2
2


In [49]:
nums = [9, 4, 8, 5, 7, 9, 10, 11]
index = binarySearch_min(nums)
print(index)

Before: left: 0, mid: 3, right: 7
3


In [53]:
# find one of lower value in an array
def binarySearch_min_v2(nums: List[int]) -> int:
    left = 0
    right = len(nums) - 1
    
    while left < right:
        mid = left + ((right - left) >> 1)
        print(f"Before: left: {left}, mid: {mid}, right: {right}")
        if nums[mid] > nums[mid + 1]:
            left = mid + 1
        else:
            right = mid
        print(f"After: left: {left}, mid: {mid}, right: {right}")
    return left
    
nums = [9, 8, 4, 5, 7, 9, 10, 11]
index = binarySearch_min_v2(nums)
print(index)

Before: left: 0, mid: 3, right: 7
After: left: 0, mid: 3, right: 3
Before: left: 0, mid: 1, right: 3
After: left: 2, mid: 1, right: 3
Before: left: 2, mid: 2, right: 3
After: left: 2, mid: 2, right: 2
2


In [59]:
# Find one peek value in array
def binarysearch_max_v1(nums: List[int]) -> int:
    left = 0
    right = len(nums) - 1
    
    while left < right:
        
        mid = left + ((right - left) >> 1)
        print(f"Before: left: {left}, mid: {mid}, right: {right}")
        if nums[mid] < nums[mid + 1]:
            left = mid + 1
        else:
            right = mid
        
        print(f"After: left: {left}, mid: {mid}, right: {right}")
    return left
        
nums = [4, 5, 7, 9, 10, 1, 2, 3] 
print(binarysearch_max_v1(nums))

Before: left: 0, mid: 3, right: 7
After: left: 4, mid: 3, right: 7
Before: left: 4, mid: 5, right: 7
After: left: 6, mid: 5, right: 7
Before: left: 6, mid: 6, right: 7
After: left: 7, mid: 6, right: 7
7


In [65]:
def binarysearch_max_v2(nums: List[int]) -> int:
    left = 0
    right = len(nums) - 1
    
    while left < right:
        mid = left + ((right - left) >> 1)
        print(f"Before: left: {left}, mid: {mid}, right: {right}")
        if nums[mid] > nums[mid - 1] and nums[mid] > nums[mid + 1]:
            return mid
        elif nums[mid] < nums[mid + 1]:
            left = mid + 1
        else:
            right = mid
        
        print(f"After: left: {left}, mid: {mid}, right: {right}")
    return left if nums[left] > nums[left - 1] and left == len(nums) - 1 else -1
        
nums = [4, 5, 7, 9, 10, 1, 2, 3] 
print(binarysearch_max_v2(nums))

Before: left: 0, mid: 3, right: 7
After: left: 4, mid: 3, right: 7
Before: left: 4, mid: 5, right: 7
After: left: 6, mid: 5, right: 7
Before: left: 6, mid: 6, right: 7
After: left: 7, mid: 6, right: 7
7


# 4. Comments
通常使用Binary Search解决问题，首先第一步要考虑的收索空间的设置，即：$[left, right]$ or $[left, right)$, 这个空间的构造可以从以下两个方面如下：

In [69]:
# 1. 从提供的数据数组上考虑设置，比如上面的模板
def binarySearch(nums: List[int], target: int) -> int:
    left = 0
    right = len(nums) - 1
    
    while left <= right:
        mid = left + ((right - left) >> 1)
        if nums[mid] == target:
            return mid
        elif nums[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1

arr = [1, 2, 3, 4, 5, 6, 7, 8, 9]
index = binarySearch(arr, 3)
print(index)  

2


In [70]:
# 2. 自己构造一下收索空间。比如sqrt的问题。
class Solution:
    def mySqrt(self, x: int) -> int:
        
        left = 0
        right = x
        
        while left < right:
            
            mid = left + ((right - left) >> 1)
            
            mid_sqrt = mid ** 2
            
            if mid_sqrt < x and (mid + 1) ** 2 > x:
                return mid
            elif mid_sqrt < x:
                left = mid + 1
            else:
                right = mid
        return left
Solution().mySqrt(10)

3